In [1]:
pip install langchain langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 945.2/945.2 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.3/999.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.5/640.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: zstandard
    Found existing installation: zstandard 0.22.0
    Uninstalling zstandard-0.22.0:
      Successfully uninstalled zstandard-0.22.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.6.0
    Uninstalling tiktoken-0.6.0:
      Successfully uninstalled tiktoken-0.6.0
  Attempting uninstall: pydantic-core
    Found existin

In [23]:
!pip install python-dotenv
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
import langchain


## API Key Setup and Verification

In [45]:
load_dotenv()

True

In [8]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_response = llm.invoke("Tell me a joke")
print(llm_response)

content="Why don't scientists trust atoms? \n\nBecause they make up everything!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--ee431a7d-77ea-48f8-b69a-dc3dc9a8a6e2-0' usage_metadata={'input_tokens': 4, 'output_tokens': 17, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}}


## Output Parser

In [9]:
from langchain_core.output_parsers import StrOutputParser

In [11]:
output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Why don't scientists trust atoms? \n\nBecause they make up everything!"

## Simple Chain


In [12]:
chain = llm | output_parser

In [13]:
chain.invoke("Tell me something about sun")

"The Sun is a G-type main-sequence star,  meaning it's a yellow dwarf that generates its energy through nuclear fusion of hydrogen into helium in its core.  This process releases enormous amounts of energy in the form of light and heat, which is what makes life on Earth possible.  It's about 93 million miles (150 million kilometers) from Earth, and its gravity holds our entire solar system together."

## Structured Output


In [19]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone  '\n'")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.
Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.
Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output


MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Insane camera, especially at night', 'Solid battery life'], cons=['Pricey', 'No charger included', 'New button layout'], summary='Great phone, but a few annoying quirks keep it from being perfect.')